In [1]:
import pickle

with open("../19_tree/processed_data/cluster_similarities.pkl" , "rb") as f:
    similarity_dict = pickle.load(f)

print(similarity_dict.keys())

dict_keys(['kl_divergence', 'bhattacharyya_coefficient', 'mahalanobis_distance'])


In [15]:
print(len(similarity_dict["mahalanobis_distance"].keys()))
print(list(similarity_dict["mahalanobis_distance"].keys())[0])


391170
(115760, 115763)


In [16]:
import numpy as np

def create_similarity_matrix_from_dict(similarity_dict):
    """
    (cluster_id, cluster_id) -> similarity の辞書から類似度行列を作成する。

    Args:
        similarity_dict (dict): クラスタIDのペアをキーとし、類似度を値とする辞書。

    Returns:
        tuple: (similarity_matrix (np.array), cluster_id_to_index (dict))
               - 類似度行列
               - クラスタIDと行列インデックスの対応マップ
    """
    
    # 1. クラスタIDの抽出とインデックスへのマッピング
    cluster_ids = set()
    for id1, id2 in similarity_dict.keys():
        cluster_ids.add(id1)
        cluster_ids.add(id2)
        
    sorted_ids = sorted(list(cluster_ids))
    N = len(sorted_ids)
    
    # ID -> インデックスのマッピング辞書
    id_to_index = {id: i for i, id in enumerate(sorted_ids)}
    
    # 2. 類似度行列 (N x N) の初期化
    similarity_matrix = np.zeros((N, N), dtype=float)
    
    # 3. 辞書の値を行列に格納し、対称性を確保
    for (id1, id2), similarity in similarity_dict.items():
        i = id_to_index[id1]
        j = id_to_index[id2]
        
        # 値をセット (元のペア)
        similarity_matrix[i, j] = similarity
        
        # 対称な位置にもセット (類似度行列は対称である必要がある)
        if i != j:
            similarity_matrix[j, i] = similarity

    # 4. 対角成分 (自己類似度) を 1.0 に設定 (辞書にない場合を考慮)
    np.fill_diagonal(similarity_matrix, 1.0)
             
    return similarity_matrix, id_to_index

similarity_matrix, cluster_id_to_index = create_similarity_matrix_from_dict(similarity_dict["mahalanobis_distance"])

In [ ]:
import numpy as np
from sklearn.manifold import MDS
import plotly.graph_objects as go
from matplotlib.colors import hsv_to_rgb # HSVからRGBへの変換に使用

def visualize_similarity_projection_3d(similarity_matrix):
    """
    類似度行列をMDSで3次元に射影し、3D HSV色空間 (H, S, V) に配置された
    クラスタを可視化します。

    Args:
        similarity_matrix (np.array): N x N のクラスタ類似度行列。
                                     (0.0: 非類似, 1.0: 完全に類似 を想定)
    
    Returns:
        list: 最終的に次元削減されたデータに使用する色のリスト (HEX/RGB形式)。
    """
    N = similarity_matrix.shape[0]
    cluster_labels = [f"Cluster {i+1}" for i in range(N)]

    # 1. 類似度行列から距離行列への変換
    # 類似度(S)が高ければ距離(D)が小さくなるように D = 1 - S
    distance_matrix = 1 - similarity_matrix
    
    # 2. MDSによる3次元への射影 (n_components=3)
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    
    # 座標 (X1, X2, X3) を取得
    coords_3d = mds.fit_transform(distance_matrix)
    
    # ストレス値の表示 (MDSによる次元削減の忠実度を示す)
    print(f"MDS Stress Value (忠実度): {mds.stress_:.4f} (値が小さいほど良い)")
    print("-" * 40)

    # 3. 3D HSV色空間へのマッピングのための正規化
    # 3D座標の各軸を [0, 1] の範囲に正規化
    
    # X1軸を Hue (H) にマッピング
    H_input = (coords_3d[:, 0] - coords_3d[:, 0].min()) / (coords_3d[:, 0].max() - coords_3d[:, 0].min())
    
    # X2軸を Saturation (S) にマッピング
    S_input = (coords_3d[:, 1] - coords_3d[:, 1].min()) / (coords_3d[:, 1].max() - coords_3d[:, 1].min())
    
    # X3軸を Value (V) にマッピング
    V_input = (coords_3d[:, 2] - coords_3d[:, 2].min()) / (coords_3d[:, 2].max() - coords_3d[:, 2].min())
    
    # HSV (Hue, Saturation, Value) タプルを作成
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    
    # HSV を RGB に変換 (Plotlyで使うために)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # 4. マッピング結果の可視化 (3D HSV空間)
    
    fig = go.Figure(data=[go.Scatter3d(
        x=H_input,
        y=S_input,
        z=V_input,
        mode='markers+text',
        marker=dict(
            size=10,
            color=hex_colors, # 割り当てられた色でマーカーを着色
            opacity=0.8
        ),
        text=cluster_labels,
        textfont=dict(color='black'),
        textposition='top center'
    )])

    # レイアウトの設定
    fig.update_layout(
        title='クラスタ類似度プロファイルの3D HSV空間への射影',
        scene=dict(
            xaxis_title='正規化MDS-X1 (色相 H)',
            yaxis_title='正規化MDS-X2 (彩度 S)',
            zaxis_title='正規化MDS-X3 (明度 V)',
            xaxis=dict(range=[0, 1]),
            yaxis=dict(range=[0, 1]),
            zaxis=dict(range=[0, 1])
        ),
        margin=dict(r=0, l=0, b=0, t=40)
    )

    fig.show()
    
    # 各クラスタの色情報を出力
    print("\n--- クラスタの色情報 ---")
    for i in range(N):
        print(f"{cluster_labels[i]}: H={H_input[i]:.3f}, S={S_input[i]:.3f}, V={V_input[i]:.3f} -> Color: {hex_colors[i]}")
        
    # **最終的な可視化で使用する色リストを返す**
    return hex_colors





# 可視化の実行
final_cluster_colors_3d = visualize_similarity_projection_3d(similarity_matrix)
print("\n最終的に次元削減されたデータに使用する色のリスト:")
print(final_cluster_colors_3d)

# heatmap
import plotly.express as px
fig = px.imshow(similarity_matrix,
                labels=dict(x="Clusters", y="Clusters", color="Similarity"),
                x=[f"Cluster {i+1}" for i in range(similarity_matrix.shape[0])],
                y=[f"Cluster {i+1}" for i in range(similarity_matrix.shape[0])],
                color_continuous_scale='Viridis',
                text_auto=True)
fig.show()

In [ ]:
def visualize_similarity_projection_3d_improved(similarity_matrix, scaling_type='linear'):
    # ... (ステップ 1, 2 は同じ) ...
    # 距離行列の計算とMDS実行
    N = similarity_matrix.shape[0]
    cluster_labels = [f"Cluster {i+1}" for i in range(N)]
    distance_matrix = 1 - similarity_matrix
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    coords_3d = mds.fit_transform(distance_matrix)
    print(f"MDS Stress Value (忠実度): {mds.stress_:.4f}")
    print("-" * 40)
    
    # 3. 3D HSV色空間へのマッピングのための正規化 (改善)
    
    def normalize_and_scale(coords, scaling_type):
        """座標軸を[0, 1]に正規化し、スケーリングを適用する内部関数"""
        
        # 線形正規化
        normed = (coords - coords.min()) / (coords.max() - coords.min())
        
        if scaling_type == 'sqrt':
            # 中央付近の値を強調するために平方根変換を試みる（0-1の範囲で）
            normed = np.sqrt(normed)
        elif scaling_type == 'power':
            # 外側の値を強調するためにべき乗変換を試みる（0-1の範囲で）
            normed = np.power(normed, 2)
            
        return normed

    # スケーリングオプションの適用
    H_input = normalize_and_scale(coords_3d[:, 0], scaling_type)
    S_input = normalize_and_scale(coords_3d[:, 1], scaling_type)
    V_input = normalize_and_scale(coords_3d[:, 2], scaling_type)
    
    # ... (ステップ 3, 4 は同じ: HSV変換とPlotly可視化) ...
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # ... (Plotly 可視化コードは変更なし) ...
    fig = go.Figure(data=[go.Scatter3d(
        x=H_input, y=S_input, z=V_input, mode='markers+text',
        marker=dict(size=10, color=hex_colors, opacity=0.8),
        text=cluster_labels, textfont=dict(color='black'), textposition='top center'
    )])

    # レイアウトの設定
    fig.update_layout(
        title=f'クラスタ類似度プロファイルの3D HSV空間への射影 ({scaling_type} Scaling)',
        scene=dict(
            xaxis_title='正規化MDS-X1 (色相 H)',
            yaxis_title='正規化MDS-X2 (彩度 S)',
            zaxis_title='正規化MDS-X3 (明度 V)',
            xaxis=dict(range=[0, 1]),
            yaxis=dict(range=[0, 1]),
            zaxis=dict(range=[0, 1])
        ),
        margin=dict(r=0, l=0, b=0, t=40)
    )

    fig.show()

    return hex_colors

# --- 実行例 ---
# サンプルデータ (類似度行列) ... (省略、以前のものを使用) ...

# 1. デフォルト (線形) で試す
final_colors_linear = visualize_similarity_projection_3d_improved(similarity_matrix, scaling_type='linear')

# 2. 平方根変換で試す (中央の差を強調し、より広い色域を使うことを試みる)
final_colors_sqrt = visualize_similarity_projection_3d_improved(similarity_matrix, scaling_type='sqrt')

In [21]:
from scipy.stats import scoreatpercentile

def visualize_similarity_projection_3d_robust(similarity_matrix):
    # ... (ステップ 1, 2 は同じ) ...
    N = similarity_matrix.shape[0]
    cluster_labels = [f"Cluster {i+1}" for i in range(N)]
    distance_matrix = 1 - similarity_matrix
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    coords_3d = mds.fit_transform(distance_matrix)
    print(f"MDS Stress Value (忠実度): {mds.stress_:.4f}")
    print("-" * 40)
    
    # --- 3. 3D HSV色空間へのマッピングのための正規化 (ロバストスケーリング適用) ---
    
    def robust_scale_to_hsv_range(coords):
        """
        座標軸を5th/95thパーセンタイルに基づいてロバストにスケーリングし、
        [0, 1] にクリップする関数。
        """
        # 5パーセンタイルと95パーセンタイルを計算
        p5 = scoreatpercentile(coords, 5)
        p95 = scoreatpercentile(coords, 95)
        
        # スケーリング
        scaled_coords = (coords - p5) / (p95 - p5)
        
        # [0, 1] の範囲にクリップ (極端な外れ値を端に固定する)
        # これにより、外れ値は0や1に固定され、残りのデータが引き延ばされる
        clipped_coords = np.clip(scaled_coords, 0, 1)
        
        return clipped_coords

    # ロバストスケーリングの適用
    H_input = robust_scale_to_hsv_range(coords_3d[:, 0])
    S_input = robust_scale_to_hsv_range(coords_3d[:, 1])
    V_input = robust_scale_to_hsv_range(coords_3d[:, 2])
    
    # --- (以降、HSV変換とPlotly可視化コードは変更なし) ---
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # Plotly 可視化
    fig = go.Figure(data=[go.Scatter3d(
        x=H_input, y=S_input, z=V_input, mode='markers+text',
        marker=dict(size=10, color=hex_colors, opacity=0.8),
        text=cluster_labels, textfont=dict(color='black'), textposition='top center'
    )])

    # レイアウト設定
    fig.update_layout(
        title='クラスタ類似度プロファイルの3D HSV空間への射影 (ロバストスケーリング)',
        scene=dict(
            xaxis_title='正規化MDS-X1 (色相 H)',
            yaxis_title='正規化MDS-X2 (彩度 S)',
            zaxis_title='正規化MDS-X3 (明度 V)',
            xaxis=dict(range=[0, 1]),
            yaxis=dict(range=[0, 1]),
            zaxis=dict(range=[0, 1])
        ),
        margin=dict(r=0, l=0, b=0, t=40)
    )

    fig.show()

    print("\n--- クラスタの色情報 ---")
    for i in range(N):
        print(f"Cluster {i+1}: H={H_input[i]:.3f}, S={S_input[i]:.3f}, V={V_input[i]:.3f} -> Color: {hex_colors[i]}")

    return hex_colors

# 実行時には、similarity_matrixを引数として渡してください
final_colors_robust = visualize_similarity_projection_3d_robust(similarity_matrix)

MDS Stress Value (忠実度): 2718223934.2557
----------------------------------------



--- クラスタの色情報 ---
Cluster 1: H=0.533, S=0.486, V=0.486 -> Color: rgb(63, 112, 123)
Cluster 2: H=0.531, S=0.491, V=0.490 -> Color: rgb(63, 113, 124)
Cluster 3: H=0.535, S=0.487, V=0.487 -> Color: rgb(63, 111, 124)
Cluster 4: H=1.000, S=0.000, V=1.000 -> Color: rgb(255, 255, 255)
Cluster 5: H=0.529, S=0.491, V=0.490 -> Color: rgb(63, 114, 124)
Cluster 6: H=0.000, S=0.000, V=0.807 -> Color: rgb(205, 205, 205)
Cluster 7: H=0.555, S=0.000, V=1.000 -> Color: rgb(255, 255, 255)
Cluster 8: H=0.535, S=0.491, V=0.489 -> Color: rgb(63, 111, 124)
Cluster 9: H=0.664, S=1.000, V=0.000 -> Color: rgb(0, 0, 0)
Cluster 10: H=0.579, S=0.508, V=0.245 -> Color: rgb(30, 47, 62)
Cluster 11: H=0.531, S=0.491, V=0.491 -> Color: rgb(63, 113, 125)
Cluster 12: H=0.529, S=0.486, V=0.486 -> Color: rgb(63, 113, 123)
Cluster 13: H=0.233, S=0.000, V=0.794 -> Color: rgb(202, 202, 202)
Cluster 14: H=0.533, S=0.492, V=0.488 -> Color: rgb(63, 112, 124)
Cluster 15: H=0.176, S=0.765, V=0.324 -> Color: rgb(79, 82, 19)
Cluste

In [ ]:
import numpy as np
from sklearn.manifold import MDS
import plotly.graph_objects as go
from matplotlib.colors import hsv_to_rgb 
from scipy.stats import scoreatpercentile

# --- 1. 類似度辞書から行列を作成する関数 (変更なし) ---
def create_similarity_matrix_from_dict(similarity_dict):
    cluster_ids = set()
    for id1, id2 in similarity_dict.keys():
        cluster_ids.add(id1)
        cluster_ids.add(id2)
        
    sorted_ids = sorted(list(cluster_ids))
    N = len(sorted_ids)
    
    id_to_index = {id: i for i, id in enumerate(sorted_ids)}
    similarity_matrix = np.zeros((N, N), dtype=float)
    
    for (id1, id2), similarity in similarity_dict.items():
        i = id_to_index[id1]
        j = id_to_index[id2]
        similarity_matrix[i, j] = similarity
        if i != j:
            similarity_matrix[j, i] = similarity

    np.fill_diagonal(similarity_matrix, 1.0)
    return similarity_matrix, id_to_index

# --- 2. ステップ別可視化関数 ---

def visualize_projection_stepwise(similarity_matrix, scaling_type='linear'):
    """
    類似度行列をMDSで3D座標に変換し、指定されたスケーリングを適用して可視化する。

    Args:
        similarity_matrix (np.array): N x N のクラスタ類似度行列。
        scaling_type (str): 'linear' (線形正規化) または 'robust' (ロバストスケーリング)。
    """
    N = similarity_matrix.shape[0]
    cluster_labels = [f"ID {i}" for i in range(N)] # IDは行列インデックス

    # MDSの実行
    distance_matrix = 1 - similarity_matrix
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    coords_3d = mds.fit_transform(distance_matrix)
    print(f"MDS Stress Value: {mds.stress_:.4f}")
    
    # --- スケーリングの選択 ---
    
    def apply_scaling(coords, type):
        if type == 'linear':
            # 標準の線形正規化 (外れ値の影響を強く受ける)
            min_val = coords.min()
            max_val = coords.max()
            normed = (coords - min_val) / (max_val - min_val)
            return normed
        
        elif type == 'robust':
            # ロバストスケーリング (P5-P95に基づく引き延ばし)
            p5 = scoreatpercentile(coords, 5)
            p95 = scoreatpercentile(coords, 95)
            scaled = (coords - p5) / (p95 - p5)
            # [0, 1]にクリップして外れ値を端に固定
            clipped = np.clip(scaled, 0, 1) 
            return clipped
        else:
            raise ValueError("scaling_typeは'linear'または'robust'である必要があります。")

    # 3. 座標のスケーリングとHSVマッピング
    H_input = apply_scaling(coords_3d[:, 0], scaling_type)
    S_input = apply_scaling(coords_3d[:, 1], scaling_type)
    V_input = apply_scaling(coords_3d[:, 2], scaling_type)
    
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # 4. 可視化
    fig = go.Figure(data=[go.Scatter3d(
        x=H_input, y=S_input, z=V_input, mode='markers+text',
        marker=dict(size=10, color=hex_colors, opacity=0.8),
        text=cluster_labels, textfont=dict(color='black'), textposition='top center'
    )])

    fig.update_layout(
        title=f'MDS射影結果のHSV空間マッピング ({scaling_type.capitalize()} Scaling)',
        scene=dict(
            xaxis_title='H (Hue) - MDS X1', yaxis_title='S (Saturation) - MDS X2', zaxis_title='V (Value) - MDS X3',
            xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 1]), zaxis=dict(range=[0, 1])
        )
    )
    fig.show()
    
    return H_input, S_input, V_input




# --- 実行ステップ ---

## ステップ 1: MDS + 線形正規化 (デフォルトの動作)
# 外れ値 '3' が範囲を支配し、中心に点が集まる傾向を確認します。
print("\n--- 実行 1: 線形正規化 ---")
H_linear, S_linear, V_linear = visualize_projection_stepwise(similarity_matrix, scaling_type='linear')

## ステップ 2: MDS + ロバストスケーリング
# 外れ値の影響を減らし、中央の塊が引き延ばされてHSV全体を使う様子を確認します。
print("\n--- 実行 2: ロバストスケーリング (P5-P95) ---")
H_robust, S_robust, V_robust = visualize_projection_stepwise(similarity_matrix, scaling_type='robust')

In [ ]:
import numpy as np
from sklearn.manifold import MDS
import plotly.graph_objects as go
from matplotlib.colors import hsv_to_rgb 
from scipy.stats import scoreatpercentile
import warnings

# --- 1. 類似度辞書から行列を作成する関数 (省略: 変更なし) ---
# ... (create_similarity_matrix_from_dict 関数はそのまま使用します) ...

# --- 2. ステップ別可視化関数 (外れ値フィルタリング機能追加) ---

def visualize_projection_stepwise(similarity_matrix, scaling_type='linear', filter_clipped=False):
    """
    類似度行列をMDSで3D座標に変換し、指定されたスケーリングを適用して可視化する。
    ロバストスケーリング適用時、クリップされた外れ値を除外可能。
    """
    N = similarity_matrix.shape[0]
    # 行列インデックスに基づいたラベルを使用
    cluster_labels = np.array([f"Index {i}" for i in range(N)]) 

    # MDSの実行
    distance_matrix = 1 - similarity_matrix
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    
    # 警告を抑制: MDSは時々収束しない警告を出すため
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        coords_3d = mds.fit_transform(distance_matrix)
    
    print(f"MDS Stress Value: {mds.stress_:.4f}")
    
    # --- スケーリングの選択 ---
    
    def apply_scaling(coords, type):
        if type == 'linear':
            min_val = coords.min()
            max_val = coords.max()
            normed = (coords - min_val) / (max_val - min_val)
            return normed
        
        elif type == 'robust':
            p5 = scoreatpercentile(coords, 5)
            p95 = scoreatpercentile(coords, 95)
            scaled = (coords - p5) / (p95 - p5)
            # ロバストスケーリングの核：クリッピング
            clipped = np.clip(scaled, 0, 1) 
            return clipped
        else:
            raise ValueError("scaling_typeは'linear'または'robust'である必要があります。")

    # 3. 座標のスケーリングとHSVマッピング
    H_input_all = apply_scaling(coords_3d[:, 0], scaling_type)
    S_input_all = apply_scaling(coords_3d[:, 1], scaling_type)
    V_input_all = apply_scaling(coords_3d[:, 2], scaling_type)
    
    # --- 4. クリップされた外れ値のフィルタリング ---
    
    if filter_clipped and scaling_type == 'robust':
        # クリップされていない点 (0 < X < 1) のインデックスを見つける
        # 許容誤差 epsilon を設定し、厳密に0や1でないものを抽出
        epsilon = 1e-6
        is_not_clipped = (H_input_all > epsilon) & (H_input_all < 1 - epsilon) & \
                         (S_input_all > epsilon) & (S_input_all < 1 - epsilon) & \
                         (V_input_all > epsilon) & (V_input_all < 1 - epsilon)
        
        H_input = H_input_all[is_not_clipped]
        S_input = S_input_all[is_not_clipped]
        V_input = V_input_all[is_not_clipped]
        filtered_labels = cluster_labels[is_not_clipped]
        
        print(f"--- フィルタリング結果 ---")
        print(f"合計クラスタ数: {N}")
        print(f"クリップされなかった（フィルタリング後）クラスタ数: {len(H_input)}")
        print(f"除外されたクラスタ（外れ値）数: {N - len(H_input)}")
        print("-" * 40)
        
    else:
        # フィルタリングしない場合はすべて使用
        H_input, S_input, V_input = H_input_all, S_input_all, V_input_all
        filtered_labels = cluster_labels
        
    # HSV -> RGB 変換
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # 5. 可視化
    fig = go.Figure(data=[go.Scatter3d(
        x=H_input, y=S_input, z=V_input, mode='markers+text',
        marker=dict(size=10, color=hex_colors, opacity=0.8),
        text=filtered_labels, textfont=dict(color='black'), textposition='top center'
    )])

    title_suffix = f" ({scaling_type.capitalize()} Scaling)"
    if filter_clipped and scaling_type == 'robust':
        title_suffix += " - 外れ値フィルタリング適用"

    fig.update_layout(
        title='MDS射影結果のHSV空間マッピング' + title_suffix,
        scene=dict(
            xaxis_title='H (Hue) - MDS X1', yaxis_title='S (Saturation) - MDS X2', zaxis_title='V (Value) - MDS X3',
            xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 1]), zaxis=dict(range=[0, 1])
        )
    )
    fig.show()
    
    return hex_colors

# --- 実行セクション ---


## ステップ 3: ロバストスケーリング + 外れ値フィルタリング
# 中央の塊だけが、タイル状に引き延ばされた状態で描画されます。
print("\n--- 実行 3: ロバストスケーリング + 外れ値フィルタリング ---")
final_colors_filtered = visualize_projection_stepwise(similarity_matrix, scaling_type='robust', filter_clipped=True)

In [2]:
# embeddingへの色付け
import numpy as np
import pandas as pd
import pickle

data_file_path = "../18_rapids/result/20251203_053328/embedding.npz"
word_file_path = "../18_rapids/result/20251203_053328/data.npz"
hdbscan_condensed_tree_file_path = "../18_rapids/result/20251203_053328/condensed_tree_object.pkl"

data = np.load(data_file_path)
word_data = np.load(word_file_path)
embedding = data['embedding'] # UMAP次元削減結果
labels = word_data['words']
print(f"Loaded embedding data from {data_file_path}, shape: {data['embedding'].shape}")

with open(hdbscan_condensed_tree_file_path, 'rb') as f:
    hdbscan_condensed_tree = pickle.load(f)

Loaded embedding data from ../18_rapids/result/20251203_053328/embedding.npz, shape: (115754, 2)


In [51]:
raw_tree = hdbscan_condensed_tree._raw_tree
leaf_rows = raw_tree[raw_tree['child_size'] == 1]
print(f'length of leaf rows: {len(leaf_rows)}')
point_cluster_map = {int(row['child']): int(row['parent']) for row in leaf_rows}
print(f"prepared point to cluster map, size: {len(point_cluster_map)}")

length of leaf rows: 115754
prepared point to cluster map, size: 115754


In [53]:
list(point_cluster_map.keys())[:10]

[1, 2, 4, 7, 9, 12, 13, 20, 22, 26]

In [57]:
hdbscan_condensed_tree._raw_tree

rec.array([(115754,      1, 0.92592937, 1),
           (115754,      2, 0.9155432 , 1),
           (115754,      4, 0.89551473, 1), ...,
           (116638,  98835, 2.5711596 , 1),
           (116638, 100993, 2.51200342, 1),
           (116638, 102893, 2.55952215, 1)],
          dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [58]:
# raw treeの中でchild が 3
raw_tree[raw_tree['child'] == 3]

rec.array([(115928, 3, 1.10274625, 1)],
          dtype=[('parent', '<i8'), ('child', '<i8'), ('lambda_val', '<f8'), ('child_size', '<i8')])

In [59]:
# childの順で
parent_ids = [point_cluster_map[i] for i in range(len(labels))]

In [60]:
df = pd.DataFrame({
"x": embedding[:,0],
"y": embedding[:,1],
"label": labels, # 元の単語ラベル
"cluster_id": parent_ids,
"noise": word_data["labels"] == -1, # ノイズポイントかどうか
"cluster_label": word_data["labels"]
})

print("DataFrame sample:")
print(df.head())
print(f"Unique clusters: {df['cluster_id'].nunique()}")

DataFrame sample:
          x         y label  cluster_id  noise  cluster_label
0 -4.065030 -4.102986    in      115756   True             -1
1 -3.943040 -3.826538   for      115754   True             -1
2 -4.072035 -4.150127    on      115754   True             -1
3 -4.072558 -4.774632   The      115928   True             -1
4 -3.993835 -4.264351  with      115754   True             -1
Unique clusters: 871


In [63]:
df_denoised = df[~df['noise']].copy()
print(f"Denoised data size: {df_denoised.shape[0]}")

Denoised data size: 6367


In [42]:
px.scatter(df_denoised, x='x', y='y',
           color=df_denoised['cluster_label'].astype(str),
           title='Denoised Embedding with Cluster Colors',
           ).update_traces(marker=dict(size=3, opacity=0.7)).update_layout(height=800, width=800).show()

c:\Users\acero\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [61]:
print(f"unique cluster IDs in denoised data: {len(df_denoised['cluster_id'].unique())}")
print(f"unique cluster labels in denoised data: {len(df_denoised['cluster_label'].unique())}")

unique cluster IDs in denoised data: 483
unique cluster labels in denoised data: 410


In [64]:
px.scatter(df_denoised, x='x', y='y',
           color=df_denoised['cluster_id'].astype(str),
           title='Denoised Embedding with Cluster Colors',
           ).update_traces(marker=dict(size=3, opacity=0.7)).update_layout(height=800, width=800).show()

c:\Users\acero\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [50]:
# 115754がparentのchildの個数

print(len(hdbscan_condensed_tree._raw_tree[hdbscan_condensed_tree._raw_tree['parent'] == 115754]))
# クラスタサイズが1かつ115754を親に持つ点の数
print(hdbscan_condensed_tree._raw_tree[(hdbscan_condensed_tree._raw_tree['child_size'] != 1) & (hdbscan_condensed_tree._raw_tree['parent'] == 115754)])

18240
[(115754, 115755, 0.94978291, 97497) (115754, 115757, 0.94978291,    19)]


In [ ]:

print(f'unique labels: {len(np.unique(data["words"]))}')
is_noise = data["words"] == -1
df_denoised = df[~is_noise]
print(f'Denoised data shape: {df_denoised.shape}')

unique labels: 411
Denoised data shape: (6367, 4)


In [5]:
# クラスタごとのサイズ例
for cluster_id, group in list(df.groupby('cluster_id'))[:10]:
    print(f"Cluster {cluster_id}: Size {len(group)}")

Cluster 115754: Size 18238
Cluster 115755: Size 5546
Cluster 115756: Size 12687
Cluster 115757: Size 19
Cluster 115758: Size 6994
Cluster 115759: Size 16
Cluster 115760: Size 12
Cluster 115761: Size 3913
Cluster 115762: Size 19
Cluster 115763: Size 6


In [6]:
# サイズの和を確認
total_size = df.shape[0]
clustered_size = df.groupby('cluster_id').size().sum()
print(f"Total points: {total_size}, Clustered points sum: {clustered_size}")

Total points: 115754, Clustered points sum: 115754


In [69]:
import numpy as np
import pandas as pd
from sklearn.manifold import MDS
import plotly.express as px
from matplotlib.colors import hsv_to_rgb
from scipy.stats import scoreatpercentile
import warnings
import plotly.graph_objects as go
# --- 類似度行列作成関数 (再掲) ---
def create_similarity_matrix_from_dict(similarity_dict):
    """(cluster_id, cluster_id) -> similarity の辞書から類似度行列を作成する。"""
    cluster_ids = set()
    for id1, id2 in similarity_dict.keys():
        cluster_ids.add(id1)
        cluster_ids.add(id2)
        
    sorted_ids = sorted(list(cluster_ids))
    N = len(sorted_ids)
    
    id_to_index = {id: i for i, id in enumerate(sorted_ids)}
    similarity_matrix = np.zeros((N, N), dtype=float)
    
    for (id1, id2), similarity in similarity_dict.items():
        i = id_to_index.get(id1)
        j = id_to_index.get(id2)
        
        # HDBSCANのノイズクラスタID (-1) が含まれている場合は無視
        if i is None or j is None:
            continue
            
        similarity_matrix[i, j] = similarity
        if i != j:
            similarity_matrix[j, i] = similarity

    np.fill_diagonal(similarity_matrix, 1.0)
    return similarity_matrix, id_to_index
# --- MDSと色割り当て関数 (3D確認機能追加) ---
def get_cluster_colors_from_similarity(similarity_matrix, id_to_index, scaling_type='robust', hsv_mapping=('H', 'S', 'V')):
    """
    類似度行列からMDSとロバストスケーリングを用いてクラスタごとの色を計算し、
    その割り当てを3D散布図で可視化する。
    """
    N = similarity_matrix.shape[0]
    
    # 逆マッピング（インデックス -> クラスタID）
    index_to_id = {v: k for k, v in id_to_index.items()}
    cluster_labels = np.array([f"ID {index_to_id[i]}" for i in range(N)])

    # MDSの実行 (省略: 以前のコードと同一)
    distance_matrix = 1 - similarity_matrix
    mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42, normalized_stress='auto')
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        coords_3d = mds.fit_transform(distance_matrix)
    
    print(f"MDS Stress Value: {mds.stress_:.4f}")

    # スケーリングロジックの定義 (省略: 以前のコードと同一)
    def apply_scaling(coords, type):
        if type == 'robust':
            p5 = scoreatpercentile(coords, 5)
            p95 = scoreatpercentile(coords, 95)
            scaled = (coords - p5) / (p95 - p5)
            clipped = np.clip(scaled, 0, 1) 
            return clipped
        elif type == 'linear':
            min_val, max_val = coords.min(), coords.max()
            return (coords - min_val) / (max_val - min_val)
        return coords

    # 座標のスケーリングとHSVマッピング
    
    # マッピング
    hsv_inputs = {}
    hsv_names = ['H', 'S', 'V']
    
    for i, hsv_comp in enumerate(hsv_mapping):
        mds_coord = coords_3d[:, i] 
        scaled_input = apply_scaling(mds_coord, scaling_type)
        hsv_inputs[hsv_comp] = scaled_input
    
    H_input = hsv_inputs['H']
    S_input = hsv_inputs['S']
    V_input = hsv_inputs['V']
    
    hsv_colors = np.stack([H_input, S_input, V_input], axis=1)
    rgb_colors = hsv_to_rgb(hsv_colors)
    hex_colors = [f'rgb({int(r*255)}, {int(g*255)}, {int(b*255)})' for r, g, b in rgb_colors]

    # --- 3D 散布図による確認 (追加部分) ---
    fig_3d = go.Figure(data=[go.Scatter3d(
        x=H_input, y=S_input, z=V_input, mode='markers+text',
        marker=dict(size=10, color=hex_colors, opacity=0.8),
        text=cluster_labels, textfont=dict(color='black'), textposition='top center'
    )])

    title_3d = f'MDS射影結果のHSV空間マッピング確認 | Map: {hsv_mapping[0]}-{hsv_mapping[1]}-{hsv_mapping[2]}'
    fig_3d.update_layout(
        title=title_3d,
        scene=dict(
            xaxis_title=f'{hsv_mapping[0]} (MDS X1)', 
            yaxis_title=f'{hsv_mapping[1]} (MDS X2)', 
            zaxis_title=f'{hsv_mapping[2]} (MDS X3)',
            xaxis=dict(range=[0, 1]), yaxis=dict(range=[0, 1]), zaxis=dict(range=[0, 1])
        ),
        margin=dict(r=0, l=0, b=0, t=40)
    )
    fig_3d.show()
    # ------------------------------------

    # クラスタID -> 色 のマッピング辞書を作成 (省略: 以前のコードと同一)
    cluster_id_to_color = {}
    for cluster_id, index in id_to_index.items():
        if index < N:
            cluster_id_to_color[cluster_id] = hex_colors[index]

    return cluster_id_to_color
# --- メイン実行関数 ---import plotly.graph_objects as go
# ... (他のインポートと関数定義はそのまま) ...

# --- メイン実行関数 (可視化部分を修正) ---
def visualize_data_with_similarity_color(df, similarity_dict_data, x_col='x', y_col='y', cluster_col='cluster_id', scaling_type='robust', hsv_mapping=('H', 'S', 'V')):
    
    # 1. 類似度行列の作成 (省略)
    similarity_matrix, id_to_index = create_similarity_matrix_from_dict(similarity_dict_data)
    
    # 2. クラスタIDごとの色を取得
    cluster_id_to_color = get_cluster_colors_from_similarity(similarity_matrix, id_to_index, scaling_type=scaling_type, hsv_mapping=hsv_mapping)
    
    # ノイズ（cluster_id = -1）用の色は灰色に設定
    NOISE_COLOR = 'rgb(180, 180, 180)'
    
    # 3. データフレームに色情報を追加 (Plotly GO では不要だが確認のため残す)
    # df['color'] = df[cluster_col].map(cluster_id_to_color)
    
    # 4. Plotly Graph Objectsによる可視化 (修正部分)
    
    fig = go.Figure()
    
    # クラスタリングされた点 (cluster_id != -1) の追加
    clustered_df = df[df[cluster_col] != -1]
    
    for cluster_id, color in cluster_id_to_color.items():
        if cluster_id == -1:
            continue # ノイズは後で処理
            
        subset = clustered_df[clustered_df[cluster_col] == cluster_id]
        
        # サイズの大きい場合は無視する
        if len(subset) > 1000:
            # print(f"Skipping cluster {cluster_id} with size {len(subset)} (too large for visualization).")
            continue

        # print(f"Plotting cluster {cluster_id} with size {len(subset)} and color {color}.")
        
        if not subset.empty:
            fig.add_trace(go.Scatter(
                x=subset[x_col],
                y=subset[y_col],
                mode='markers',
                name=f'Cluster {cluster_id}',
                marker=dict(
                    color=color, # クラスタに割り当てられた計算済みの色を適用
                    size=8,
                    # line=dict(width=0.5, color='DarkSlateGrey')
                ),
                # ホバー情報の設定
                customdata=subset[['label', cluster_col]].values,
                hovertemplate=
                    '<b>%{customdata[0]}</b><br>' +
                    'Cluster ID: %{customdata[1]}<br>' +
                    'X: %{x:.2f}<br>' +
                    'Y: %{y:.2f}<extra></extra>' +
                    'color: ' + color
            ))
    
    # # ノイズポイント (cluster_id = -1) の追加
    # noise_df = df[df[cluster_col] == -1]
    # if not noise_df.empty:
    #     fig.add_trace(go.Scatter(
    #         x=noise_df[x_col],
    #         y=noise_df[y_col],
    #         mode='markers',
    #         name='Noise (-1)',
    #         marker=dict(
    #             color=NOISE_COLOR,
    #             size=5,
    #             opacity=0.4
    #         ),
    #         customdata=noise_df[['label', cluster_col]].values,
    #         hovertemplate=
    #             '<b>%{customdata[0]}</b><br>' +
    #             'Cluster ID: %{customdata[1]}<br>' +
    #             'X: %{x:.2f}<br>' +
    #             'Y: %{y:.2f}<extra></extra>'
    #     ))
        
    fig.update_layout(
        title=f'点群の類似度着色 | Scale: {scaling_type.capitalize()} | Mapping: {hsv_mapping[0]}-{hsv_mapping[1]}-{hsv_mapping[2]}',
        template="plotly_white",
        height=800,
        width=800
    )
    fig.show()
    # save
    html_string = fig.to_html(full_html=True, include_plotlyjs='cdn')
    with open("clustered_data_plot.html", 'w', encoding='utf-8') as f:
        f.write(html_string)
    
    return df

# --- 実行時の注意 ---
# 呼び出し側では、上記修正後の visualize_data_with_similarity_color 関数を使用してください。

# 実行：ロバストスケーリングを適用して可視化
colored_df = visualize_data_with_similarity_color(df_denoised, similarity_dict["mahalanobis_distance"], scaling_type='robust')

MDS Stress Value: 2718223934.2557


In [29]:

fig = px.scatter(
    df_denoised,
    x="x",
    y="y",
    color=df_denoised["cluster_id"].astype(str)
).show()

c:\Users\acero\AppData\Local\Programs\Python\Python39\lib\site-packages\plotly\express\_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

